<a href="https://colab.research.google.com/github/TrevorSeestedt/CSCE580-Fall2024-TrevorSeestedt-Repo/blob/main/Seestedt_Quiz4_Response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1: Analyze data to solve practical problem

a) Show the data statistics – number of features and their domains; number of rows

In [6]:
import pandas as pd
from google.colab import drive
# Loading the data
drive.mount('/content/drive')

# Loading the csv files
salesData = pd.read_csv('/content/drive/MyDrive/Senior Classes/Semester_1/CSCE 580 - Artificial Intelligence/sales_data.csv')
customerData = pd.read_csv('/content/drive/MyDrive/Senior Classes/Semester_1/CSCE 580 - Artificial Intelligence/customer_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Showing the statistics for the sales data
for column in salesData.columns:

  if salesData[column].dtype in ['int64', 'float64']:
    # values for the column if its numeric
    print(f"\nStatistics for the column: {column}")
    print(f"Count: {salesData[column].count()}")
    print(f"  Min: {salesData[column].min()}")
    print(f"  Max: {salesData[column].max()}")
    print(f"  Mean: {sum(salesData[column]) / len(salesData[column]):.2f}")
    print(f"  Median: {sorted(salesData[column].dropna())[len(salesData[column]) // 2]}")

  else:
    # values for the column if it is non-numeric
    print(f"\nStatistics for the column: {column}")
    print(f"Unique Value: {salesData[column].nunique()}")
    print(f"Frequent Values: {salesData[column].mode()[0]}")



Statistics for the column: invoice_no
Unique Value: 99457
Frequent Values: I100008

Statistics for the column: customer_id
Unique Value: 99457
Frequent Values: C100004

Statistics for the column: category
Unique Value: 8
Frequent Values: Clothing

Statistics for the column: quantity
Count: 99457
  Min: 1
  Max: 5
  Mean: 3.00
  Median: 3

Statistics for the column: price
Count: 99457
  Min: 5.23
  Max: 5250.0
  Mean: 689.26
  Median: 203.3

Statistics for the column: invoice_date
Unique Value: 797
Frequent Values: 24-11-2021

Statistics for the column: shopping_mall
Unique Value: 10
Frequent Values: Mall of Istanbul


In [14]:
# Showing the statistics for the customer data
for column in customerData.columns:
    if customerData[column].dtype in ['int64', 'float64']:
        # Values for the column if it's numeric
        print(f"\nStatistics for the column: {column}")
        print(f"Count: {customerData[column].count()}")
        print(f"  Min: {customerData[column].min()}")
        print(f"  Max: {customerData[column].max()}")
        print(f"  Mean: {sum(customerData[column]) / len(customerData[column]):.2f}")
        print(f"  Median: {sorted(customerData[column].dropna())[len(customerData[column]) // 2]}")
    else:
        # Values for the column if it is non-numeric
        print(f"\nStatistics for the column: {column}")
        print(f"Unique Value: {customerData[column].nunique()}")
        print(f"Frequent Values: {customerData[column].mode()[0]}")


Statistics for the column: customer_id
Unique Value: 99457
Frequent Values: C100004

Statistics for the column: gender
Unique Value: 2
Frequent Values: Female

Statistics for the column: age
Count: 99338
  Min: 18.0
  Max: 69.0
  Mean: nan
  Median: 43.0

Statistics for the column: payment_method
Unique Value: 3
Frequent Values: Cash


b) Perform exploratory data analysis

1 - Missing Data

In [16]:
missing_data = {}
for column in salesData.columns:
    missing_count = sum(1 for value in salesData[column] if pd.isna(value))
    missing_data[column] = missing_count


print("Missing Data per Column:")
for column, count in missing_data.items():
    print(f"{column}: {count} missing values")

Missing Data per Column:
invoice_no: 0 missing values
customer_id: 0 missing values
category: 0 missing values
quantity: 0 missing values
price: 0 missing values
invoice_date: 0 missing values
shopping_mall: 0 missing values


2 - Unique Product Categories and the max items bought



In [20]:
# finding unique product categories
product_categories = set(salesData['category'])
print(f"Number of unique categories: {len(product_categories)}")

# max items bought
category_max_items = {}
for category in product_categories:
    max_items_bought = max(row['quantity'] for index, row in salesData.iterrows() if row['category'] == category)
    category_max_items[category] = max_items_bought

print("Max items bought per category:")
for category, maxItems in category_max_items.items():
    print(f"{category}: {maxItems}")

Number of unique categories: 8
Max items bought per category:
Food & Beverage: 5
Shoes: 5
Toys: 5
Souvenir: 5
Technology: 5
Cosmetics: 5
Books: 5
Clothing: 5


3 - Who buys more (Males and Females) and top 3 things bought by gender

In [27]:
customer_gender = {}
for index, row in customerData.iterrows():
  customer_gender[row['customer_id']] = row['gender']

salesData_with_gender = []
for index, row in salesData.iterrows():
  gender = customer_gender.get(row['customer_id'])
  salesData_with_gender.append({**row, 'gender': gender})

In [31]:
# total purchases by gender
gender_total = {'Male': 0,
                'Female': 0}

for row in salesData_with_gender:
  if row['gender'] in gender_total:
    gender_total[row['gender']] += row['quantity']


# print results
print("Total Purchases by Gender:")
for gender, total in gender_total.items():
    print(f"{gender}: {total}")

Total Purchases by Gender:
Male: 120053
Female: 178659


In [34]:
# top three items bought by each gender
genderItems = {'Male': {},
                'Female': {}}

for row in salesData_with_gender:
  gender = row['gender']
  category = row['category']
  quantity = row['quantity']

  if gender in genderItems:
    genderItems[gender][category] = genderItems[gender].get(category, 0) + quantity


# display
for gender, items in genderItems.items():
  topThreeItems = sorted(items.items(), key=lambda x: x[1], reverse=True)[:3]
  print(f"{gender}: {topThreeItems}")

Male: [('Clothing', 41519), ('Cosmetics', 18204), ('Food & Beverage', 17915)]
Female: [('Clothing', 62039), ('Cosmetics', 27261), ('Food & Beverage', 26362)]


4 - Most used payment method

In [38]:
paymentCounter = {}
for index, row in customerData.iterrows():
  paymentMethod = row['payment_method']
  paymentCounter[paymentMethod] = paymentCounter.get(paymentMethod, 0) + 1

most_used = max(paymentCounter, key = paymentCounter.get)
print(f"Most used payment method: {most_used}")
print(f"{paymentCounter}")

Most used payment method: Cash
{'Credit Card': 34931, 'Debit Card': 20079, 'Cash': 44447}


5 - Month with most purchases

In [39]:
# Going to have to extract the month
monthCounter = {}

for index, row in salesData.iterrows():
  date = row['invoice_date']
  month = date.split('-')[1]
  monthCounter[month] = monthCounter.get(month, 0) + 1

most_used_month = max(monthCounter, key = monthCounter.get)
print(f"Month with most purchases: {most_used_month}")
print(f"{monthCounter}")

Month with most purchases: 01
{'08': 7635, '12': 7680, '11': 7563, '05': 7697, '10': 7764, '03': 8730, '01': 11608, '07': 7877, '06': 7581, '02': 10482, '04': 7487, '09': 7353}


6 - Divide the purchases in bins of $500, get bin with the highest purchase and which is the bin with the most purchases?



In [43]:
# defining bins

bins = [0, 500, 1000, 1500, 2000, float('inf')]
labels = ['0-500', '500-1000', '1000-1500', '1500-2000', '2000+']

binCount = {label: 0 for label in labels}
binTotal = {label: 0 for label in labels}

# assigning the purchases to the bins
for index, row in salesData.iterrows():
  price = row['price']
  for i in range(len(bins) - 1):
    if bins[i] < price <= bins[i+1]:
      binCount[labels[i]] += 1
      binTotal[labels[i]] += price
      break

# bin with most amount of purchases
most_purchases_bin = max(binCount, key=binCount.get)
print(f"Bin with the most purchases: {most_purchases_bin}")

# bin with the highest total purchase
highest_total_bin = max(binTotal, key=binTotal.get)
print(f"Bin with the highest total purchase: {highest_total_bin}")

print(f"{binCount}")
print(f"{binTotal}")


Bin with the most purchases: 0-500
Bin with the highest total purchase: 2000+
{'0-500': 56781, '500-1000': 15801, '1000-1500': 9879, '1500-2000': 8950, '2000+': 8046}
{'0-500': 5621141.660000966, '500-1000': 11555800.160001934, '1000-1500': 11709936.980000887, '1500-2000': 14031801.100001296, '2000+': 25632686.0100002}


In [44]:
highest_month= [row for index, row in salesData.iterrows() if int(row['invoice_date'].split('-')[1]) == most_used_month]

# Re-bin data
highest_month_bin_counts = {label: 0 for label in labels}
for row in highest_month:
    amount = row['price']
    for i in range(len(bins) - 1):
        if bins[i] < amount <= bins[i + 1]:
            highest_month_bin_counts[labels[i]] += 1
            break

most_purchases_bin_in_highest_month = max(highest_month_bin_counts, key=highest_month_bin_counts.get)
print(f"Bin with most purchases in highest purchased month: {most_purchases_bin_in_highest_month}")

Bin with most purchases in highest purchased month: 0-500
